In [1]:
# Parameters
location = "default_location"
date = "2025-05-07"  # This will be overridden by papermill

# Ensure the date is in YYYY-MM-DD format
from datetime import datetime
date = datetime.today().strftime('%Y-%m-%d')  # Dynamically get today's date in YYYY-MM-DD format

print(f"Starting forecast process for Location: {location}, Date: {date}")

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

# Step 1: Query historical data
def useHistoricalDataQuery(location):
    print(f"Fetching historical data for location: {location}")
    # Replace this with actual database query or API call
    dates = pd.date_range(end=date, periods=365)
    temps = np.sin(np.linspace(0, 10, 365)) * 10 + 20 + np.random.randn(365)
    df = pd.DataFrame({'date': dates, 'temperature': temps})
    df.set_index('date', inplace=True)
    print(f"Historical data fetched: {df.head()}")
    return df

# Load data
data = useHistoricalDataQuery(location)
print(f"Loaded historical data:\n{data.describe()}")

# Step 2: Preprocess data
print("Starting data preprocessing...")
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[['temperature']])
print(f"Scaled data (first 5 rows):\n{scaled_data[:5]}")

# Prepare sequences
def create_sequences(data, seq_length):
    print(f"Creating sequences with sequence length: {seq_length}")
    x, y = [], []
    for i in range(len(data) - seq_length):
        x.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    print(f"Created {len(x)} sequences.")
    return np.array(x), np.array(y)

seq_length = 30
x, y = create_sequences(scaled_data, seq_length)
print(f"Input shape: {x.shape}, Output shape: {y.shape}")

# Reshape for LSTM
x = x.reshape((x.shape[0], x.shape[1], 1))
print(f"Reshaped input for LSTM: {x.shape}")

# Step 3: Build LSTM model
print("Building LSTM model...")
model = Sequential()
model.add(LSTM(50, return_sequences=False, input_shape=(seq_length, 1)))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
print("LSTM model built successfully.")

# Step 4: Train model
print("Training LSTM model...")
model.fit(x, y, epochs=10, batch_size=16, verbose=1)
print("Model training completed.")

# Step 5: Predict next 7 days
print("Generating forecast for the next 7 days...")
last_seq = scaled_data[-seq_length:]
forecast = []
input_seq = last_seq.copy()

for i in range(7):
    pred = model.predict(input_seq.reshape(1, seq_length, 1))
    forecast.append(pred[0, 0])
    input_seq = np.append(input_seq[1:], pred).reshape(seq_length, 1)
    print(f"Day {i+1} forecast: {forecast[-1]}")

# Inverse scale
forecast = scaler.inverse_transform(np.array(forecast).reshape(-1, 1)).flatten()
print(f"Forecast after inverse scaling: {forecast}")

# Output forecast values
forecast_dates = pd.date_range(start=pd.to_datetime(date) + pd.Timedelta(days=1), periods=7)
forecast_data = [{"date": str(d.date()), "temperature": float(f)} for d, f in zip(forecast_dates, forecast)]
print("Final forecast data:")
for entry in forecast_data:
    print(entry)

# Explicitly output the forecast data
forecast_data

Starting forecast process for Location: default_location, Date: 2025-05-08


Fetching historical data for location: default_location
Historical data fetched:             temperature
date                   
2024-05-09    19.765822
2024-05-10    19.669489
2024-05-11    20.118523
2024-05-12    21.641085
2024-05-13    20.055924
Loaded historical data:
       temperature
count   365.000000
mean     21.738296
std       6.794773
min       8.011087
25%      15.958041
50%      23.237447
75%      27.695998
max      32.093127
Starting data preprocessing...
Scaled data (first 5 rows):
[[0.48811209]
 [0.48411189]
 [0.50275792]
 [0.56598186]
 [0.50015851]]
Creating sequences with sequence length: 30
Created 335 sequences.
Input shape: (335, 30, 1), Output shape: (335, 1)
Reshaped input for LSTM: (335, 30, 1)
Building LSTM model...
LSTM model built successfully.
Training LSTM model...
Epoch 1/10


D:\Documents\OE\Szakdolgozat\szakdolgozat\weather-warehouse\backend\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


 1/21 ━━━━━━━━━━━━━━━━━━━━ 16s 811ms/step - loss: 0.3973

17/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3223   

...

Day 6 forecast: 0.2606394588947296


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


Day 7 forecast: 0.2589477002620697
Forecast after inverse scaling: [14.874114 14.664556 14.518391 14.417734 14.343771 14.287818 14.247076]
Final forecast data:
{'date': '2025-05-09', 'temperature': 14.874114036560059}
{'date': '2025-05-10', 'temperature': 14.664555549621582}
{'date': '2025-05-11', 'temperature': 14.518390655517578}
{'date': '2025-05-12', 'temperature': 14.417734146118164}
{'date': '2025-05-13', 'temperature': 14.343770980834961}
{'date': '2025-05-14', 'temperature': 14.28781795501709}
{'date': '2025-05-15', 'temperature': 14.247076034545898}


[{'date': '2025-05-09', 'temperature': 14.874114036560059},
 {'date': '2025-05-10', 'temperature': 14.664555549621582},
 {'date': '2025-05-11', 'temperature': 14.518390655517578},
 {'date': '2025-05-12', 'temperature': 14.417734146118164},
 {'date': '2025-05-13', 'temperature': 14.343770980834961},
 {'date': '2025-05-14', 'temperature': 14.28781795501709},
 {'date': '2025-05-15', 'temperature': 14.247076034545898}]

In [2]:
# Parameters
location = "Cairo International Airport"
